# Crawl D&C places from the Church of Jesus Christ of Latter-day Saints

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup

from models.crawl_utils import get_page, save_page

In [ ]:
# config
base = 'https://doctrineandcovenantscentral.org'
host = 'https://doctrineandcovenantscentral.org/places-of-the-doctrine-and-covenants/'
base_dir = '../data/raw/dc_places'
bs_parser = 'html.parser'
delay_seconds = 5

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    

In [ ]:
def extract_hrefs(html):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    link_tags = soup.find_all('div', {'data-column-clickable': True})

    # Extract href attribute from each anchor tag and append to the list
    for link in link_tags:
        href = link.get('data-column-clickable')
        if href:
            href = base + href
            hrefs.append(href)

    return hrefs

In [ ]:
def get_path(url):
    if not url.endswith('/'):
        url += '/'
    path_components = urlparse(url).path.split('/')
    return os.path.join(base_dir, f"{path_components[-2]}.json")

In [ ]:
dc_places_links = []

status_code, html = get_page(host, delay_seconds)
if status_code != 200:
    print("Error!", status_code)
hrefs = extract_hrefs(html)
if len(hrefs) != 0:
    dc_places_links.extend(hrefs)
    
print(dc_places_links[:5])
len(dc_places_links)

In [ ]:
for url in dc_places_links:
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    save_page(path_file,url,html)
    
print("End")